# Load Models

In the previous notebook we saved the large model. In this notebook we have a look on how to reload it.

## Setup & Data

In [10]:
# Import packages
import os
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

tf.keras.backend.set_floatx('float64')

# Define random seed for whole notebook
RSEED=42

In [11]:
# Load data
df = pd.read_csv("../data/boston.csv")

# Define target 
y = df.pop("MEDV")

# Split into train and test set
X_train, X_test, y_train, y_test = train_test_split(df, y, random_state=RSEED)

In [12]:
df

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,9.67
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,9.08
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,5.64
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,6.48


In [13]:
# Scale numerical features
# Scale numerical values
col_scale = ['CRIM', 'ZN', 'INDUS', 'NOX', 'RM', 'AGE', 'DIS', 'TAX', 'PTRATIO', 'LSTAT']

#scaler = MinMaxScaler()
scaler = StandardScaler()
X_train[col_scale] = scaler.fit_transform(X_train[col_scale])
X_test[col_scale] = scaler.transform(X_test[col_scale])

# Convert to np array
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

The SavedModel format is a directory containing a protobuf binary and a TensorFlow checkpoint. Inspect the saved model directory:

In [4]:
ls saved_model

my_large_model/


In [5]:
ls saved_model/my_large_model

assets/            keras_metadata.pb  variables/
fingerprint.pb     saved_model.pb


## SavedModel format

Reload a fresh Keras model from the saved model:

In [6]:
# Load the saved model
with tf.device('/cpu:0'):
    new_large_model = tf.keras.models.load_model('saved_model/my_large_model')

# Check its architecture
new_large_model.summary()

2026-02-05 19:22:33.636313: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2026-02-05 19:22:33.636361: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2026-02-05 19:22:33.636376: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2026-02-05 19:22:33.636455: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2026-02-05 19:22:33.636488: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 512)               6656      
                                                                 
 dense_8 (Dense)             (None, 512)               262656    
                                                                 
 dense_9 (Dense)             (None, 512)               262656    
                                                                 
 dense_10 (Dense)            (None, 512)               262656    
                                                                 
 dense_11 (Dense)            (None, 1)                 513       
                                                                 
Total params: 795137 (6.07 MB)
Trainable params: 795137 (6.07 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


The restored model is compiled with the same arguments as the original model. Try running evaluate and predict with the loaded model:

In [14]:
# Evaluate the restored model
with tf.device('/cpu:0'):
    loss, mse = new_large_model.evaluate(X_test, y_test, verbose=2)
print(f'Model MSE: {mse}')

4/4 - 0s - loss: 2.2996 - mse: 12.2858 - 19ms/epoch - 5ms/step
Model MSE: 12.285848431110336


The Model MSE is higher than in the notebook before, even though we are using the same data-split. We are using the same model and the same data. Therefore, we would assume that we also receive the same MSE. 

> **Exercise:** Can you find out where this notebook varies from the procedure in the notebook before? It might help to have a look at the values in X_train in both notebooks.

<details><summary>
Click here for a hint...
</summary>
Check out how the data were preprocessed. Did both notebooks use the same scaler?
</details>

In [ ]:
# the difference is in the scaling method.
# Notebook 03 uses StandarScaler()
# Notebook 04 uses MinMaxScaler() which gives a higher MSE